# Generate new set of instance 

We use the generation method described in the paper The Trigger Arc TSP: Optimise the Picking Process in Warehouses with Compactable Storage Systems

Cerrone, C., Truvolo, M., Dragone, R., Battaglia, R. (2025). The Trigger Arc TSP: Optimise the Picking Process in Warehouses with Compactable Storage Systems. In: Juan, A.A., Faulin, J., Lopez-Lopez, D. (eds) Decision Sciences. DSA ISC 2024. Lecture Notes in Computer Science, vol 14778. Springer, Cham. https://doi.org/10.1007/978-3-031-78238-1_26

## Desc

In their paper, Cerrone et al. do not fully describe the instance creation. We assume the points are randomly generated within their descibed 5km side square and we also assume the distances to be computed in meters

In [5]:
import math

def euclidean_dist(city1, city2) -> int:
    x1, y1 = city1
    x2, y2 = city2
    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return round(distance,2)

In [6]:
import random

def generate_cities(n: int) -> list:
    # Generate n cities with random (x, y) coordinates
    cities = []
    for _ in range(n):
        cities.append((random.uniform(0, 5000), random.uniform(0, 5000)))
    
    return cities

In [7]:
import os

def generate_instances(n: int, r: int, path: str, type:str):
    cities = generate_cities(n)
    num_arcs = n * (n - 1)  # Fully connected directed graph
    memory_dct ={} #Needed for the arc id in the relation line 

    os.makedirs(os.path.dirname(path), exist_ok=True)

    with open(path, 'w') as f:
        # Write header
        f.write(f"{n} {num_arcs} {r}\n")

        # base arcs 
        arc_id = 0
        for i in range(n):
            for j in range(i + 1, n):
                cost = euclidean_dist(cities[i], cities[j])
                f.write(f"{arc_id} {i} {j} {cost}\n")
                memory_dct[(i,j)] = (arc_id,cost)
                arc_id += 1
                f.write(f"{arc_id} {j} {i} {cost}\n")
                memory_dct[(j,i)] = (arc_id,cost)
                arc_id += 1
        
        # triggers
        for rel_id in range(r):
            start_pair = tuple(random.sample(range(n), 2))
            end_pair = tuple(random.sample(range(n), 2))

            start_arc_id, cost = memory_dct[start_pair]
            end_arc_id, _ = memory_dct[end_pair]

            if type == "balanced":
                cost = random.uniform(cost/2, 2*cost)
            elif type == "increase":
                cost = random.uniform(cost, 2*cost)
            elif type == "decrease":
                cost = random.uniform(cost/2, cost)

            cost = round(cost,2)
            
            f.write(f"{rel_id} {start_arc_id} {start_pair[0]} {start_pair[1]} {end_arc_id} {end_pair[0]} {end_pair[1]} {cost}\n")
        

In [10]:
for type in ["balanced","increase","decrease"]:    
    for n in [10,15,20,25]:
        for r in [1,2,4,8,16]:
            for i in ["0","1","2"]:
                rel = int(r*n**2)
                path = "./../../instances/instances_generic/" + type + "_n_" + str(n) + "_r_" + str(rel) + "_" + i
                generate_instances(n,rel,path,type)